# Задача о рекламных компаниях

Компания запустила три рекламные кампании (A, B, C) и хочет оценить их влияние на ключевые бизнес-метрики. У тебя есть данные о пользователях, их взаимодействиях с сайтом и покупках. 

**Таблицы:**
1. Лог посещений сайта (site_visits)
2. Действия пользователей (user_actions)
3. Покупки (purchases)

In [111]:
import pandas as pd
import numpy as np

np.random.seed(42)
n_users = 6000
campaigns = ['A', 'B', 'C']
site_visits = pd.DataFrame({
    'visit_id': np.arange(1, n_users + 1),
    'user_id': np.arange(1001, 1001 + n_users),
    'campaign': np.random.choice(campaigns, n_users),
    'visit_timestamp': pd.date_range(start='2024-02-01', periods=n_users, freq='T')
})

actions = ['view', 'add_to_cart', 'purchase']
n_actions = 10000
user_actions = pd.DataFrame({
    'action_id': np.arange(1, n_actions + 1),
    'user_id': np.random.choice(site_visits['user_id'], n_actions),
    'action': np.random.choice(actions, n_actions, p=[0.7, 0.2, 0.1]),
    'action_timestamp': pd.date_range(start='2024-02-01', periods=n_actions, freq='T')
})

purchase_users = user_actions[user_actions['action'] == 'purchase']['user_id'].unique()
n_purchases = len(purchase_users)
purchases = pd.DataFrame({
    'order_id': np.arange(2001, 2001 + n_purchases),
    'user_id': purchase_users,
    'campaign': site_visits.set_index('user_id').loc[purchase_users, 'campaign'].values,
    'amount': np.random.uniform(50, 500, n_purchases).round(2),
    'items_count': np.random.randint(1, 5, n_purchases),
    'purchase_timestamp': pd.date_range(start='2024-02-01', periods=n_purchases, freq='T')
})

/var/folders/jm/lyl2x71d4p3019zwc7l2sdm40000gn/T/ipykernel_13051/12038745.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'visit_timestamp': pd.date_range(start='2024-02-01', periods=n_users, freq='T')
/var/folders/jm/lyl2x71d4p3019zwc7l2sdm40000gn/T/ipykernel_13051/12038745.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'action_timestamp': pd.date_range(start='2024-02-01', periods=n_actions, freq='T')
/var/folders/jm/lyl2x71d4p3019zwc7l2sdm40000gn/T/ipykernel_13051/12038745.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'purchase_timestamp': pd.date_range(start='2024-02-01', periods=n_purchases, freq='T')


In [113]:
site_visits.head()

,visit_id,user_id,campaign,visit_timestamp
0,1,1001,C,2024-02-01 00:00:00
1,2,1002,A,2024-02-01 00:01:00
2,3,1003,C,2024-02-01 00:02:00
3,4,1004,C,2024-02-01 00:03:00
4,5,1005,A,2024-02-01 00:04:00


In [115]:
user_actions.head()

,action_id,user_id,action,action_timestamp
0,1,5320,view,2024-02-01 00:00:00
1,2,1784,view,2024-02-01 00:01:00
2,3,1950,view,2024-02-01 00:02:00
3,4,4940,add_to_cart,2024-02-01 00:03:00
4,5,1045,add_to_cart,2024-02-01 00:04:00


In [117]:
purchases.head()

,order_id,user_id,campaign,amount,items_count,purchase_timestamp
0,2001,5567,C,429.18,4,2024-02-01 00:00:00
1,2002,4499,A,496.83,1,2024-02-01 00:01:00
2,2003,4749,C,214.86,2,2024-02-01 00:02:00
3,2004,1278,C,254.15,1,2024-02-01 00:03:00
4,2005,2877,A,370.25,3,2024-02-01 00:04:00


**1. Определить, какая рекламная кампания привлекла больше всего пользователей.**

In [120]:
count_visit_users = site_visits.groupby('campaign', as_index=False)['user_id'].value_counts()
group_A_visits = count_visit_users.query('campaign == "A"')['count'].sum()
group_B_visits = count_visit_users.query('campaign == "B"')['count'].sum()
group_C_visits = count_visit_users.query('campaign == "C"')['count'].sum()
print(f'Campaign A visits: {group_A_visits}, campaign B visits: {group_B_visits}, campaign C visits: {group_C_visits}')

Campaign A visits: 2052, campaign B visits: 1972, campaign C visits: 1976


Компания А привлекла больше всего пользователей, если смотреть только на посещение сайта по рекламе

**2. Рассчитать конверсию в покупку для каждой кампании.**

In [124]:
buying = purchases.groupby('campaign', as_index=False).value_counts()
camp_A_buy = buying.query('campaign == "A"')['count'].sum()
camp_B_buy = buying.query('campaign == "B"')['count'].sum()
camp_C_buy = buying.query('campaign == "C"')['count'].sum()
print(f'Campaign A buyers: {camp_A_buy}, campaign B buyers: {camp_B_buy}, campaign C buyers: {camp_C_buy}')

Campaign A buyers: 345, campaign B buyers: 310, campaign C buyers: 303


Теперь мы можем посчитать конверсию в покупку: сколько процентов людей покупают нашу продукцию

In [127]:
A_percents = round((camp_A_buy / group_A_visits) * 100, 2)
B_percents = round((camp_B_buy / group_B_visits) * 100, 2)
C_percents = round((camp_C_buy / group_C_visits) * 100, 2)
print(f'Conversion to purchase, group A: {str(A_percents) + '%'}, conversion to purchase, group B: {str(B_percents) + '%'}, conversion to purchase, group C: {str(C_percents) + '%'}')

Conversion to purchase, group A: 16.81%, conversion to purchase, group B: 15.72%, conversion to purchase, group C: 15.33%


 Так же можем сказать, что конверсия в покупку у рекламной кампании А выше

**3. Проанализировать средний чек и среднее количество товаров в заказе.**

In [131]:
mean_receipt_A = round(purchases.query('campaign == "A"').amount.mean(), 2)
mean_receipt_B = round(purchases.query('campaign == "B"').amount.mean(), 2)
mean_receipt_C = round(purchases.query('campaign == "C"').amount.mean(), 2)
print(f'Mean receipt A: {mean_receipt_A}, mean receipt B: {mean_receipt_B}, mean receipt C: {mean_receipt_C}')
mean_count_items_A = round(purchases.query('campaign == "A"').items_count.mean(), 1)
mean_count_items_B = round(purchases.query('campaign == "B"').items_count.mean(), 1)
mean_count_items_C = round(purchases.query('campaign == "C"').items_count.mean(), 1)
print(f'Mean items A: {mean_count_items_A}, mean items B: {mean_count_items_B}, mean items C: {mean_count_items_C}')

Mean receipt A: 271.06, mean receipt B: 282.8, mean receipt C: 283.68
Mean items A: 2.5, mean items B: 2.4, mean items C: 2.5


Средний чек группы А ниже чем в остальных случаях, но среднее количество вещей в заказе в каждой группе примерно одинаково

**4. Определить, есть ли статистически значимые различия в метриках между кампаниями.**

In [135]:
from scipy.stats import f_oneway
f_oneway(purchases.query('campaign == "A"').amount, purchases.query('campaign == "B"').amount,purchases.query('campaign == "C"').amount)

F_onewayResult(statistic=0.9163976798894942, pvalue=0.40030863270060635)

Больше всего нас интересует средний чек, ведь именно от него зависит, насколько прибыльным будет наш бизнес. Применив ANOVA-тест мы получили, что p-value > 0.05 а это значит, что статистически значимых различий между компаниями нет